## Preprocessing the Dataset

In [40]:
import os
import pandas as pd

In [50]:
def update_csv_file(file_path, states_lt):
    
    for state, path in zip(states_lt, file_path):
        df = pd.read_csv(path)

        # add the state column 
        df.insert(loc=0, column='State', value=state)

        # save the csv
        df.to_csv(path, index=False)

In [51]:


# read the file in the folder
path = os.listdir('./redbus_details/')
file_path = [ os.path.join("./redbus_details/", file) for file in path]

list_of_states = ["Andhra Pradesh", "Assam", "Himachal Pradesh", "Goa", "Kerala", "Rajasthan",
                                           "South Bengal",  "Telangana", "Uttar Pradesh", "West Bengal"]
# update the state column
update_csv_file(file_path, list_of_states)


## Preprocessing and merge the all data into single file

In [52]:

df = [ pd.read_csv(file) for file in file_path]

# concatenate the all dataframe 
df_comb = pd.concat(df, ignore_index=True)

# Remove the INR in the price column
df_comb['Price'] = df_comb['Price'].str.replace('INR', '')

# Extract the numeric value in the seat_availability column
df_comb['Seat_Availability'] = df_comb['Seat_Availability'].str.extract(r"(\d+)")


# Add the id column to the dataset at first
df_comb.insert(loc=0, column='id', value=range(1, len(df_comb)+1))

df_comb.to_csv("redbus_routes.csv", index=False)

## Store the data into database

In [53]:
from sqlite3 import connect

conn = connect('redbus_db.db')
curr = conn.cursor()

In [61]:
import pandas as pd
df = pd.read_csv('./redbus_routes.csv')

In [62]:
# create a table
table = """
    CREATE TABLE redbus_details (
        id INT PRIMARY KEY,
        state VARCHAR(100),
        bus_name VARCHAR(255),
        route_name VARCHAR(255),
        bus_type VARCHAR(255),
        departing_time VARCHAR(150),
        duration VARCHAR(100),
        reaching_time VARCHAR(150),
        star_rating  DECIMAL(3,2),
        price DECIMAL(10,2),
        seat_availability INT,
        route_link VARCHAR(255)  
    );
"""

In [66]:
# curr.execute("DROP TABLE IF EXISTS redbus_details;")

In [67]:
curr.execute(table)

In [68]:
# insert the data into the table
sql_query = """
    INSERT INTO redbus_details (
        id, state, bus_name, route_name, bus_type, departing_time,
        duration, reaching_time, star_rating, price, seat_availability, route_link
    ) VALUES 
    (:id,:state, :bus_name, :route_name, :bus_type, :departing_time,
    :duration, :reaching_time, :star_rating, :price, :seat_availability, :route_link);
"""



    
for index, row in df.iterrows():
    params = {
        'id': row['id'],
        'state' : row['State'],
        'bus_name': row['Bus_Name'],
        'route_name': row['Route_Name'],
        'bus_type': row['Bus_Type'],
        'departing_time': row['Departing_Time'],
        'duration': row['Duration'],
        'reaching_time': row['Reaching_Time'],
        'star_rating': row['Star_Rating'],
        'price': row['Price'],
        'seat_availability': row['Seat_Availability'],
        'route_link': row['Route_Link']
    }
    curr.execute(sql_query, params)
    conn.commit()
    

In [64]:
# curr.execute("delete from  redbus_details;")

In [70]:
dd = pd.read_sql("select * from redbus_details;", conn)
dd[dd['state']== 'Assam']['route_name'].unique()

array(['Tezpur to Guwahati', 'Guwahati to Tezpur',
       'Guwahati to Nagaon (Assam)', 'Nagaon (Assam) to Guwahati',
       'Goalpara to Guwahati', 'Jorhat to North Lakhimpur',
       'Dhubri to Guwahati', 'Jorhat to Dibrugarh',
       'North Lakhimpur to Sibsagar', 'Guwahati to Dhubri',
       'Dibrugarh to Jorhat', 'Jorhat to Tinsukia', 'Tezpur to Dibrugarh',
       'North Lakhimpur to Tezpur', 'Guwahati to Biswanath Charali',
       'North Lakhimpur to Dibrugarh', 'Tinsukia to Jorhat',
       'Tezpur to North Lakhimpur', 'Guwahati to Gohpur',
       'Guwahati to Kaliabor', 'Dibrugarh to Tezpur',
       'Biswanath Charali to Guwahati', 'Dibrugarh to North Lakhimpur',
       'Tumuki (Tezpur Medical) to Guwahati',
       'Biswanath Charali to Dibrugarh', 'Gohpur to Guwahati',
       'North Lakhimpur to Guwahati', 'Haflong to Guwahati',
       'Tezpur to Moran', 'Moran to Tezpur', 'Bokakhat to Dibrugarh',
       'North Lakhimpur to Nagaon (Assam)', 'Guwahati to Golaghat',
       'Dibru